In [ ]:
!pip install -U langchain langchain-community langchain-huggingface huggingface_hub transformers
!pip install langchain langchain-community langchain-huggingface faiss-cpu
!pip install pandas huggingface_hub tiktoken
!pip install transformers accelerate sentence-transformers langchain faiss-cpu




In [ ]:
!pip install bitsandbytes accelerate transformers peft
!pip install -U bitsandbytes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_path = '/content/drive/MyDrive/medical_chatbot/archive.zip'
import zipfile
import os

extract_path = '/content/drive/MyDrive/medical_chatbot'  # temp local folder in Colab
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/medical_chatbot/ai-medical-chatbot.csv')

df.dropna(subset=["Description", "Patient", "Doctor"], inplace=True)
df.head()

,Description,Patient,Doctor
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...


In [ ]:
from langchain.schema import Document

documents = []

for idx, row in df.head(5).iterrows():
    full_convo = f"""Question: {row['Description'].strip()}
Patient: {row['Patient'].strip()}
Doctor: {row['Doctor'].strip()}"""

    doc = Document(
        page_content=full_convo,
        metadata={"row_id": idx}
    )
    documents.append(doc)


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define the splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # max characters per chunk
    chunk_overlap=100,  # overlap between chunks
    separators=["\n\n", "\n", ".", " ", ""]  # try to split cleanly
)

# Split documents
split_docs = text_splitter.split_documents(documents)

print(f"Original docs: {len(documents)} → Split docs: {len(split_docs)}")


Original docs: 5 → Split docs: 23


In [ ]:
!pip install -U bitsandbytes
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

device = "cuda" if torch.cuda.is_available() else "cpu"

base_name = "microsoft/Phi-3.5-mini-instruct"
lora_name = "syubraj/Phi-3.5-mini-instruct-MedicalChat-QLoRA"

tokenizer = AutoTokenizer.from_pretrained(base_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, lora_name)
model = model.merge_and_unload().to(device)

def generate_response(user_query, max_length=512):
    system_msg = ("You are an empathetic, accurate medical assistant—prioritize safety and clarity.")
    prompt = (f"<|system|> {system_msg} <|end|>\n"
              f"<|user|> {user_query} <|end|>\n"
              f"<|assistant|>")
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=max_length)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("<|assistant|>")[-1].split("<|end|>")[0].strip()

print(generate_response("I have a headache and slight fever, what should I do?"))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/765 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/50.4M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


You are an empathetic, accurate medical assistant—prioritize safety and clarity.  I have a headache and slight fever, what should I do?  If you're experiencing a headache accompanied by a slight fever, here are some steps you should consider to ensure your safety and well-being:

1. **Rest**: Your body needs energy to fight off whatever is causing your symptoms.

2. **Hydration**: Drink plenty of fluids to stay hydrated. Water, herbal teas, and clear broths are good choices.

3. **Monitor your symptoms**: Keep track of your fever and headache. If your fever is above 101°F (38.3°C), or if your headache worsens, seek medical attention.

4. **Over-the-counter medication**: You may consider taking acetaminophen (Tylenol) or ibuprofen (Advil, Motrin) to help reduce fever and alleviate headache pain. However, always follow the recommended dosage on the package and consult with a healthcare provider if you have any concerns.

5. **Seek medical advice**: If your symptoms persist for more than 

In [ ]:
'''from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
'''

'from langchain.embeddings import HuggingFaceEmbeddings\n\nembedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")\n'

In [ ]:
'''# 1. Upgrade pip
!pip install --upgrade pip

# 2. Install llama-cpp-python with GPU support (prebuilt wheels)
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.79 --upgrade --no-cache-dir
'''

'# 1. Upgrade pip\n!pip install --upgrade pip\n\n# 2. Install llama-cpp-python with GPU support (prebuilt wheels)\n!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.79 --upgrade --no-cache-dir\n'

In [ ]:
'''from huggingface_hub import hf_hub_download

# Example: Meta's LLaMA-2 7B chat in Q4_K_M quantization
model_path = hf_hub_download(
    repo_id="TheBloke/Llama-2-7B-Chat-GGUF",  # Quantized model repo
    filename="llama-2-7b-chat.Q4_K_M.gguf"    # Choose the quantized file
)
'''

'from huggingface_hub import hf_hub_download\n\n# Example: Meta\'s LLaMA-2 7B chat in Q4_K_M quantization\nmodel_path = hf_hub_download(\n    repo_id="TheBloke/Llama-2-7B-Chat-GGUF",  # Quantized model repo\n    filename="llama-2-7b-chat.Q4_K_M.gguf"    # Choose the quantized file\n)\n'

In [ ]:
'''from llama_cpp import Llama

llm_cpp = Llama(
    model_path=model_path,
    n_ctx=2048,        # context length
    n_threads=8,       # adjust based on Colab CPU cores
    temperature=0.7,   # creativity
    top_p=0.95,
    repeat_penalty=1.1
)
'''

'from llama_cpp import Llama\n\nllm_cpp = Llama(\n    model_path=model_path,\n    n_ctx=2048,        # context length\n    n_threads=8,       # adjust based on Colab CPU cores\n    temperature=0.7,   # creativity\n    top_p=0.95,\n    repeat_penalty=1.1\n)\n'

In [ ]:
'''output = llm_cpp(
    "You are a helpful assistant. Explain world ar 2 in simple terms.",
    max_tokens=200,
    stop=["</s>"]
)
print(output["choices"][0]["text"])
'''

'output = llm_cpp(\n    "You are a helpful assistant. Explain world ar 2 in simple terms.",\n    max_tokens=200,\n    stop=["</s>"]\n)\nprint(output["choices"][0]["text"])\n'

In [ ]:
from langchain.vectorstores import FAISS
# Get raw text from each chunk
texts = [doc.page_content for doc in split_docs]

# Get embeddings from API (this may take a few seconds depending on length)
embeddings = embedding_model.embed_documents(texts)
# Build the vectorstore
vectorstore = FAISS.from_documents(
    documents=split_docs,
    embedding=embedding_model
)


In [ ]:
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate.from_template("""
You are an intelligent assistant.

Instructions:
1. First, decide if the question is medical/health-related or not.
2. If the question is medical/health-related:
   - Check if the retrieved context is relevant to the question.
   - If relevant, answer using the retrieved context.
   - If not relevant, answer using your verified internal medical knowledge.
   - If neither provides enough information, say: "I'm not certain based on the provided information."
   - Do not guess, assume, or fabricate.
   - Respond in one concise, medically sound sentence.
   - Always mention whether the answer comes from retrieved context or your trusted medical knowledge.
3. If the question is NOT medical:
   - Ignore the medical instructions entirely.
   - If the retrieved context is irrelevant to the question, ignore it.
   - Answer using your general knowledge in a short, factual sentence.

Chat History:
{chat_history}

Retrieved Context:
{context}

User Question:
{question}

Answer:
""")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# 1️⃣ Load Phi-3 instruct
model_id = "microsoft/phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

# 2️⃣ Create pipeline for inference
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.2,
    top_p=0.9
)

# 3️⃣ Wrap in LangChain LLM
llm = HuggingFacePipeline(pipeline=pipe)

# 4️⃣ Create memory & retriever
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Use your existing FAISS retriever or any other retriever here
retriever = vectorstore.as_retriever()

# 5️⃣ Build conversation chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)

# 6️⃣ Simple chat loop
while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        break
    result = qa_chain.invoke({"question": query})
    print("Bot:", result["answer"])


In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory

llm = HuggingFacePipeline(pipeline=mistral_pipeline)

retriever = vectorstore.as_retriever(search_type="mmr",
    search_kwargs={"k": 3,"score_threshold": 0.8}  # fetch top 6 docs instead of 4
)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)


In [ ]:

while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        break
    response = qa_chain.invoke(query)
    print("Bot:", response)
